In [13]:
%reload_ext autoreload
%autoreload 2

import mlflow
import os
import numpy as np
import pandas as pd
import pendulum
import sys
import xgboost as xgb

from loguru import logger
from pathlib import Path
from pycaret import regression
from sklearn.metrics import make_scorer, mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor


sys.path.append(str(Path.cwd().parent))
sys.path.append(os.path.join(str(Path.cwd().parent), "src"))
from settings.params import *
from metrics import eval_metrics
from pipeline import define_pipeline
from plot import prediction_error_plot, residual_plot
from tracking import mlflow_log_search

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)

In [14]:
data = pd.read_csv(CLEANED_DATA)
TARGET_NAME = MODEL_PARAMS['TARGET_NAME']

In [ ]:
mlflow.set_tracking_uri(uri="http://localhost:8080")

# Modeling


## Train/Test Split


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data, data[TARGET_NAME], test_size=MODEL_PARAMS["TEST_SIZE"], random_state=SEED)

logger.info(f"\nX train: {x_train.shape}\nY train: {y_train.shape}\n"
            f"X test: {x_test.shape}\nY test: {y_test.shape}")

## Training


In [ ]:
df = x_train.copy()
df[TARGET_NAME] = np.log(y_train)

In [ ]:
exp_reg = regression.setup(df, target=TARGET_NAME, max_encoding_ohe=200, log_experiment=True, experiment_name="building-energy-prediction-training", train_size=0.8)
regression.set_config('seed', SEED)

# Removing useless metrics improve training speed
regression.remove_metric('MAPE')
regression.remove_metric('MSE')
regression.remove_metric('RMSLE')

In [ ]:
best_threes_model = regression.compare_models(n_select=3)

In [ ]:
mlflow.end_run()

L'entrainement sur plusieurs types de modèles avec PyCaret montre que pour ce problème, les modèles: ExtraTreesRegressor, XGBRegressor et RandomForestRegressor sont les plus adaptés. Toutefois, nous pensons que les performances obtenus lors de ce premier entrainement peuvent être nettement améliorées. Nous allons attendre de faire le réglage de paramètres et d'obtenir les modèles finaux avant de faire un choix.


## Fine-Tuning


In [ ]:
ESTIMATOR_PARAMS = {
    ExtraTreesRegressor.__name__: {
        "estimator": ExtraTreesRegressor(),
        "params": {
            'regressor__estimator__n_estimators': np.arange(10, 200, 5),
        }
    },
    RandomForestRegressor.__name__: {
        "estimator": RandomForestRegressor(),
        "params": {
            'regressor__estimator__n_estimators': np.arange(10, 200, 5),
        }
    },
    xgb.XGBRegressor.__name__: {
        "estimator": xgb.XGBRegressor(),
        "params": {
            'regressor__estimator__n_estimators': np.arange(10, 200, 5),
        }
    }
}

In [ ]:
CURRENT_DATE = pendulum.now()

search_cvs = {}

def rmse(actual, predicted):
    return np.sqrt(mean_squared_error(actual, predicted))

scoring = {'r2': make_scorer(r2_score),
          'rmse': make_scorer(rmse, greater_is_better=False),
          'mae': make_scorer(mean_absolute_error, greater_is_better=False)}

# Create an experiment if not exists
exp_name = "building-energy-prediction-tuning-sklearn"
experiment = mlflow.get_experiment_by_name(exp_name)
if not experiment:
    experiment_id = mlflow.create_experiment(exp_name)
else:
    experiment_id = experiment.experiment_id

with mlflow.start_run(run_name=f"Session-{CURRENT_DATE.strftime('%Y%m%d_%H%m%S')}", experiment_id=experiment_id) as parent_run:
    for estimator_name, settings in ESTIMATOR_PARAMS.items():
        with mlflow.start_run(run_name=estimator_name, nested=True, experiment_id=experiment_id):  
            estimator = settings["estimator"]
            param_grid = settings["params"]
            pipeline = define_pipeline(numerical_transformer=[SimpleImputer(strategy="median"), RobustScaler()],
                            categorical_transformer=[SimpleImputer(strategy="constant", fill_value="undefined"), OneHotEncoder(drop="if_binary", handle_unknown="ignore")],
                            target_transformer=True,
                            estimator=estimator
                        ) 
            grid_search = GridSearchCV(
                estimator=pipeline,  # Instantiate the estimator
                param_grid=param_grid,
                scoring=scoring,
                refit='r2',
                cv=5,  # Adjust the number of cross-validation folds as needed
                n_jobs=-1  # Use all available cores
            )
            grid_search.fit(x_train, y_train)
            search_cvs[estimator_name] = grid_search

            mlflow.log_param("Estimator", estimator_name)
            mlflow_log_search(grid_search)
mlflow.end_run()

## Model Evaluation on Test Data


We are going to evaluate the fine-tuned models to see which one we are going to pick as the final model.


In [ ]:
# Obtaining best_models after fine-tuning
models = { f"{estimator_name}": search_cv.best_estimator_ for estimator_name, search_cv in search_cvs.items()}

In [ ]:
exp_name = "building-energy-prediction-evaluation"
experiment = mlflow.get_experiment_by_name(exp_name)
if not experiment:
    experiment_id = mlflow.create_experiment(exp_name)
else:
    experiment_id = experiment.experiment_id

def evaluate_models(estimators, x_train, x_test, y_train, y_test):
    # Dict of R2 scores for the estimators
    r2_scores = {}
    with mlflow.start_run(run_name=f"Session-{CURRENT_DATE.strftime('%Y%m%d_%H%m%S')}", experiment_id=experiment_id):
        for estimator_name, estimator in estimators.items():
            with mlflow.start_run(run_name=estimator_name, nested=True, experiment_id=experiment_id): 
                y_train_pred = estimator.predict(x_train)
                y_test_pred = estimator.predict(x_test)

                train_metrics = eval_metrics(y_train, y_train_pred)
                test_metrics = eval_metrics(y_test, y_test_pred)

                # Add the R2 score of the model to the global dict
                r2_scores[estimator_name] = test_metrics['r2']

                # Log the regressor parameters
                mlflow.log_params(estimator.regressor.steps[-1][1].get_params())

                # Log the best metric
                mlflow.log_metrics(test_metrics)

                # Log the model
                mlflow.sklearn.log_model(estimator.best_estimator_, "model")

                logger.info(f"""{estimator_name} performance \n{pd.DataFrame({'train': train_metrics, 'test': test_metrics}).T}""")
    return max(r2_scores.items(), key=lambda item: item[1])

mlflow.end_run()

best_estimator, score = evaluate_models(models, x_train, x_test, y_train, y_test)

logger.info(f"""{best_estimator} is the best estimator found for this problem with an R2 score of {score}""")

### Prediction Error Plot


In [ ]:
prediction_error_plot(models, x_train, x_test, y_train, y_test)

### Residual Plot


In [ ]:
residual_plot(models, x_train, x_test, y_train, y_test)